<a href="https://colab.research.google.com/github/olwflynn/Learning/blob/master/Learning%20Physics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-dev set to manually installed.
software-properties-common is already the newest version (0.96.24.32.18).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  glew-utils
The following NEW packages will be installed:
  libgl1-mesa-glx libglew-dev libglew2.0 libosmesa6 libosmesa6-dev
0 upgraded, 5 newly installed, 0 to remove and 21 not upgraded.
Need to get 2,916 kB of archives.
After this operation, 12.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgl1-mesa-glx amd64 20.0.8-0ubuntu1~18.04.1 [5,532 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libglew2.0 amd64 2.0.0-5 [140 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/univ

In [2]:
!pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install free-mujoco-py mujoco

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 KB 25.1 MB/s eta 0:00:00


**Now you need to restart the runtime as numpy is apparently automatically imported...**

In [ ]:
import mujoco_py
import gym

Compiling /usr/local/lib/python3.8/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.8/dist-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8/usr
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8/usr/local
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8/usr/local/lib
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuexten

In [ ]:
import numpy as np

# def plt_update(ax, colors=['b']):
#     x = np.linspace(0,1,100)
#     if ax.lines:
#         for line in ax.lines:
#             line.set_xdata(x)
#             y = np.random.random(size=(100,1))
#             line.set_ydata(y)
#     else:
#         for color in colors:
#             y = np.random.random(size=(100,1))
#             ax.plot(x, y, color)
#     fig.canvas.draw()


def live_plot(data_dict, figsize=(7,5), title=''):
    # clear_output(wait=True)
    plt.figure(figsize=figsize)
    # for label,data in data_dict.items():
    obs = data_dict["obs"]
    preds = data_dict["preds"]
    print("obs: ", obs, "\n preds: ",preds)

    plt.plot(obs, label="obs")
    plt.plot(range(len(obs)-1, len(obs)+len(preds)-1), preds, label="preds")
    
    plt.title(title)
    plt.grid(True)
    plt.xlabel('epoch')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();

Need to restart runtime if cant find 'mujuco'

In [ ]:
env = gym.make("InvertedDoublePendulum-v4", new_step_api=True)
env.reset(seed=42)
env.step(env.action_space.sample())

In [ ]:
# naive forecaster where we predict the angle at the next 3 steps is the same as the current angle

def naive_forecaster(X):
    # X is the all data known to the forecaster at the time. X is only the current and previous angles for now.
    
    current_angle = X[-1]
    angle_t_plus_1 = current_angle
    angle_t_plus_2 = current_angle
    angle_t_plus_3 = current_angle
    
    return np.array([angle_t_plus_1, angle_t_plus_2, angle_t_plus_3])


In [ ]:
# get forecast error given a predicted trajectory and observations

def forecast_error(X_obs, preds):
    
    mse = sum(np.square(X_obs - preds)) / len(X_obs)
    
    return mse

In [ ]:
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

# observation, info = env.reset(seed=42)
obs_history = []
# fig,ax = plt.subplots(1,1)
# ax.set_xlabel('t')
# ax.set_ylabel('obs_idx_1')
# ax.set_xlim(0,20)
# ax.set_ylim(-5,5)

for _ in range(10):
    print("epoch #", _)
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
    print(observation, reward, terminated, truncated, info)
    obs_history.append(observation[1])
    # plt.plot(obs_history[0][0])
    # plt.show()
    preds = obs_history[-1]+[0, 0.2,0.2,0.2]
    # obs_concat_preds = obs_history
    # for pred in preds:
    #     obs_concat_preds.append(pred)
    live_plot({"obs": obs_history, "preds": preds}, title="Radians by epoch")
    time.sleep(1)
    if terminated or truncated:
        print(obs_history)
        print("Prediction error in last 3 steps: ", forecast_error(obs_history[-3:], preds[-3:]))
        break
        # obs_history = []
        # observation, info = env.reset()

env.close()

The episode ends when any of the following happens:

Truncation: The episode duration reaches 1000 timesteps.

Termination: Any of the state space values is no longer finite.

Termination: The absolutely value of the vertical angle between the pole and the cart is greater than 0.2 radian. i.e. 11.4592 degrees

1; vertical angle of the pole on the cart; -Inf; Inf; angle (rad)

*JTBD*

- Run thousands of episodes to generate training data with features (current and prev angles and other data) and labels (next few angles)
- Train forecast model to predict trajectory of angle
- Start episode with control over speed i.e. time lag between steps
- Predict expected time to end of episode at step t with measure of uncertainty
- Predict trajectory of angle in next few time steps at step t
- At end of episode output prediction errors at a few time steps


In [ ]:
!pip install torch

In [ ]:
import torch
X = torch.randn(1, 15)
print(X)

In [ ]:
y = torch.ones(1, 10)
print(y)

In [ ]:
# y = softmax(layer2(layer1(X)))

from torch import nn

class TestNetwork(nn.Module):
  def __init__(self):
    super(TestNetwork, self).__init__()
    self.seqlayers = nn.Sequential(
        nn.Linear(15, 20),
        nn.ReLU(),
        nn.Linear(20, 10),
        nn.ReLU(),
    )
    self.softmax = nn.Softmax(dim=1)

  def forward(self, X):
    hidden1 = self.seqlayers(X)
    output = self.softmax(hidden1)
    return output



In [ ]:
model = TestNetwork()
print(model)

In [ ]:
preds = model(X)
preds

In [ ]:
preds.size()

In [ ]:
loss_ = torch.nn.functional.binary_cross_entropy(preds, y)
loss_

In [ ]:
# Backpropagation for one step
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

optimizer.zero_grad()
loss_.backward()
optimizer.step()

new_preds = model(X)
new_loss = torch.nn.functional.binary_cross_entropy(new_preds, y)

print(new_loss)

In [ ]:
def create_rows_for_episode(ep_num):
  env.reset(seed=42)
  for _ in range(100):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
    features = np.array(observation[:8]).reshape(1, 8)
    if _ == 0:
      rows_for_ep = features
    else:
      rows_for_ep = np.concatenate((rows_for_ep, features), axis=0)
    if terminated or truncated:
      labels = np.array(range(_, -1,-1)).reshape(_+1, 1)
      ep_num_arr = np.array([ep_num for i in range(len(labels))]).reshape(_+1, 1)
      assert len(labels) == len(rows_for_ep)
      rows_for_ep = np.concatenate((rows_for_ep, labels), axis=1)
      rows_for_ep = np.concatenate((rows_for_ep, ep_num_arr), axis=1)
      # print(len(rows_for_ep), type(rows_for_ep))
      return rows_for_ep
  
    

In [ ]:
# create a dataset with positions of the pendulum as features and time to terminate as label

import pandas as pd

columns = []
for i in range(8):
  columns.append(f'feature_{i}')
columns.append('time_to_terminate')
columns.append('ep_num')

#create array
num_episodes = 10000
for j in range(num_episodes):
  episode_rows = create_rows_for_episode(j)
  if j == 0:
    arr = episode_rows
  else:
    arr = np.concatenate([arr, episode_rows], axis=0)
env.close()

df = pd.DataFrame(arr, columns=columns)
print(df.head())
print(df.describe())
print(df.info())

In [ ]:
# terminating conditions of the episodes

df_terminated = df[df['time_to_terminate']==0]
print(df_terminated.head())
print(df_terminated.describe())
print(df_terminated.info())

In [ ]:
# plot a five of the episodes against various features

ep_split_df = [df[df['ep_num']==i] for i in range(5)]
num_features = 8
fig, axs = plt.subplots(num_features, figsize=(12, 8))
plt.suptitle('First 5 episodes against features')
for i in range(num_features):
  for el in ep_split_df:
    ep = el['ep_num'].values[0]
    axs[i].plot(el[f'feature_{i}'], label=f'episode_{int(ep)}')
    axs[i].set_title(f'feature_{i}', fontsize='small')
# plt.legend(loc='right')
plt.show()

In [ ]:
from random import triangular
# prepare dataset for train, validate, test
# use final 20% episodes for testing

test_size = 0.2*num_episodes
valid_size = 0.1*num_episodes
train_size = num_episodes - (test_size+valid_size)

test_df = df[df['ep_num']<=test_size]
valid_df = df[(df['ep_num']>test_size)&(df['ep_num']<=(valid_size+test_size))]
train_df = df[(df['ep_num']>(valid_size+test_size))]

features = []
for i in range(8):
  features.append(f'feature_{i}')

test_labels = test_df['time_to_terminate']
test_features = test_df[features]

valid_labels = valid_df['time_to_terminate']
valid_features = valid_df[features]

train_labels = train_df['time_to_terminate']
train_features = train_df[features]

print('Test dataset shapes:', test_labels.shape, test_features.shape)
print('Validation dataset shapes:', valid_labels.shape, valid_features.shape)
print('Train dataset shapes:', train_labels.shape, train_features.shape)

In [ ]:
# linear regression model to predict time_to_terminate using features as baseline
# this model does not take into account the time series i.e. it assumes that t, t+1 are independent

from sklearn.linear_model import LinearRegression

regr = LinearRegression()

regr.fit(train_features, train_labels)

preds = regr.predict(test_features)

print('MSE: \n', forecast_error(test_labels, preds))
print('Score: \n', regr.score(test_features, test_labels))

In [ ]:
print(regr.coef_)
print(regr.intercept_)

Results:

> n=100, mse=2.07, model=lin reg
coef= [ 1.33349380e+00 -6.43072005e-01 -1.25702564e+00  6.48821446e+00
  5.30884445e+00 -1.92036934e-01 -8.90950510e-02 -3.15731262e-03]
intercept= -7.1225544550046775



> n=10000, mse=2.07, model=lin reg
coef=[ 8.05432692  6.7247005   2.55556171  4.98903376  5.38681353 -0.09618963
  0.021658    0.03276889]
intercept=-6.228216297398191





In [ ]:
# log linear so only positve? 
# look at gamma regression to use linear regression and only predict positive values

from sklearn.linear_model import PoissonRegressor

pois_regr = PoissonRegressor()

pois_regr.fit(train_features, train_labels)

pois_preds = pois_regr.predict(test_features)
print('Poisson model')
print('MSE: \n', forecast_error(test_labels, pois_preds))
print('Score: \n', pois_regr.score(test_features, test_labels))

In [ ]:
from sklearn.linear_model import TweedieRegressor

tweed_regr = TweedieRegressor(power=1.5)

tweed_regr.fit(train_features, train_labels)

tweed_preds = tweed_regr.predict(test_features)
print('Tweedie w/ power=1.5')
print('MSE: \n', forecast_error(test_labels, tweed_preds))
print('Score: \n', tweed_regr.score(test_features, test_labels))

In [ ]:
# take linear model but set preds less than zero to 0.
preds_positive = preds.copy()
preds_positive[preds_positive < 0] =  0

print('Positive only predictions linear model')
print('MSE: \n', forecast_error(test_labels, preds_positive))


In [ ]:
# wants to learn the x^2 features so linear is a bad model e.g. direction (+/-) does not matter for falling for pendulum
# implement NN to add non-linearity - use softplus for output

class DenseNet(nn.Module):
  def __init__(self):
    super(DenseNet, self).__init__()
    self.layer1 = nn.Linear(8,20)
    self.layer2 = nn.Linear(20,1)
    self.softplus = nn.Softplus()
  
  def forward(self, X):
    hidden1 = self.layer1(X)
    hidden2 = self.layer2(hidden1)

    output = self.softplus(hidden2)
    return output

In [ ]:
dense_model = DenseNet()
print(dense_model)

In [ ]:
# Test fwd pass of NN
X_fwd_pass = torch.ones(1, 8, dtype=torch.float32, device='cpu')
dense_model(X_fwd_pass)

In [ ]:
# preprocess data into pytorch tensors

dtype = torch.float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device: \n', device)

train_labels_tensor = torch.tensor(train_labels.values, dtype=dtype, device=device).view(len(train_labels), -1)
train_features_tensor = torch.tensor(train_features.values, dtype=dtype, device=device, requires_grad=True)

valid_labels_tensor = torch.tensor(valid_labels.values, dtype=dtype, device=device).view(len(valid_labels), -1)
valid_features_tensor = torch.tensor(valid_features.values, dtype=dtype, device=device)

test_labels_tensor = torch.tensor(test_labels.values, dtype=dtype, device=device).view(len(test_labels), -1)
test_features_tensor = torch.tensor(test_features.values, dtype=dtype, device=device)

print('Test tensor shapes:', test_labels_tensor.shape, test_features_tensor.shape)
print('Validation tensor shapes:', valid_labels_tensor.shape, valid_features_tensor.shape)
print('Train tensor shapes:', train_labels_tensor.shape, train_features_tensor.shape)

In [ ]:
from sklearn.metrics import log_loss
# Training loop

def train(y_train, X_train, y_valid, X_valid, y_test, X_test, model_, \
          model_name, epochs=10000, learning_rate=0.1):
  criterion = nn.MSELoss()
  optimizer = torch.optim.SGD(model_.parameters(), lr=learning_rate)
  model_.name = model_name
  for epoch in range(epochs):
    optimizer.zero_grad()

    train_preds = model_(X_train)
    train_loss = torch.sqrt(criterion(train_preds, y_train))
    if epoch % 500 == 0:
      valid_preds = model_(X_valid)
      valid_loss = torch.sqrt(criterion(valid_preds, y_valid))
      print(f'epoch #{epoch} \n training loss: \n', train_loss.item(), \
            '\n validation loss: \n', valid_loss.item(), '\n')
    train_loss.backward()
    optimizer.step()

  test_preds = model_(X_test)
  test_loss = torch.sqrt(criterion(test_preds, y_test))

  print('Final metrics: \n training loss: ', train_loss.item(), \
        '\n validation loss: ', valid_loss.item(),'\n test loss: ', \
        test_loss.item())


In [ ]:
# model to gpu and train
dense_model.to(device)

train(train_labels_tensor, train_features_tensor, valid_labels_tensor, valid_features_tensor, \
      test_labels_tensor, test_features_tensor, dense_model, model_name='10000eps_DenseNet_10000epochs', epochs=10000)

In [ ]:
# next add reLU for additional non-linearity

class DenseNetRelu(nn.Module):
  def __init__(self):
    super(DenseNetRelu, self).__init__()
    self.layer1 = nn.Linear(8,20)
    self.relu1 = nn.ReLU()
    self.layer2 = nn.Linear(20,1)
    self.relu2 = nn.ReLU()
    self.softplus = nn.Softplus()
  
  def forward(self, X):
    hidden1 = self.relu1(self.layer1(X))
    hidden2 = self.relu2(self.layer2(hidden1))

    output = self.softplus(hidden2)
    return output

In [ ]:
dense_plus_relu_model = DenseNetRelu()
print(dense_plus_relu_model)

In [ ]:
dense_plus_relu_model.to(device)

train(train_labels_tensor, train_features_tensor, valid_labels_tensor, valid_features_tensor, \
      test_labels_tensor, test_features_tensor, dense_plus_relu_model, model_name='10000eps_DenseNetRelu_10000epochs', epochs=10000)

In [ ]:
class LSTMnet(nn.Module):
  def __init__(self, batch_first=True):
    super(LSTMnet, self).__init__()
    self.lstm = nn.LSTM(8, 16)
    self.linear = nn.Linear(16, 1)
    self.softplus = nn.Softplus()
  
  def forward(self, ep_features):
    # embeds.view(len(sentence), 1, -1)) --> ??
    batch_size, ep_length, num_feats = ep_features.size()
    # used to be ep_features.view(len(ep_features),1,-1)
    lstm_out, _ =  self.lstm(ep_features.view(batch_size, -1, num_feats))
    # may also need unpad the upper zeroes
    # lstm_out, _ = self.lstm(ep_features.view(ep_features.size()[0], ep_features.size()[1],1,-1))
 # used to be ep_features.view(len(ep_features), -1)
    linear_out = self.linear(lstm_out.view(batch_size, -1, 16))
    output = self.softplus(linear_out)

    return output



In [ ]:
lstm_model = LSTMnet()
lstm_model.to(device)

In [ ]:
# prepare data for lstm i.e. [(ep1_features, ep1_labels), (ep2_features, ep2_labels), ...] in tensors

eps = []

for ep_num in range(num_episodes):
  episode = df[df['ep_num']==ep_num]
  episode_feats = torch.tensor(episode[features].values, dtype=dtype, device=device, requires_grad=True)
  episode_labs = torch.tensor(episode['time_to_terminate'].values, dtype=dtype, device=device).view(len(episode),-1)
  eps.append((episode_feats, episode_labs))

assert len(eps) == num_episodes

# split into train, validate, test
test_episodes = eps[:int(test_size)]
valid_episodes = eps[int(test_size): int(test_size)+int(valid_size)]
train_episodes = eps[int(test_size)+int(valid_size):]

print(len(train_episodes), train_episodes[:10])

assert len(test_episodes)+ len(valid_episodes)+len(train_episodes) == len(eps)

In [ ]:
# batch processing
from torch.utils.data import Dataset

class EpisodesDataset(Dataset):
    def __init__(self, episode_feats, episode_labs):
        self.episode_feats = episode_feats
        self.episode_labs = episode_labs

    def __len__(self):
        return len(self.episode_labs)

    def __getitem__(self, idx):
        return self.episode_feats[idx], self.episode_labs[idx]



In [ ]:
def ep_data_to_pytorch_dataset(ep_data):
  all_episode_feats = [ep_data[i][0] for i in range(len(ep_data))]
  all_episode_labs = [ep_data[i][1] for i in range(len(ep_data))]
  pytorch_dataset = EpisodesDataset(all_episode_feats, all_episode_labs)
  return pytorch_dataset

train_dataset = ep_data_to_pytorch_dataset(train_episodes)
valid_dataset = ep_data_to_pytorch_dataset(valid_episodes)
test_dataset = ep_data_to_pytorch_dataset(test_episodes)

print(len(train_dataset), len(valid_dataset), len(test_dataset))
train_dataset[:2]

In [ ]:
# find max ep length

MAX_EP_LENGTH = 1
for ep in eps:
  if len(ep[0])>MAX_EP_LENGTH:
    MAX_EP_LENGTH = len(ep[0])
print(MAX_EP_LENGTH)

In [ ]:
def ep_data_to_padded_pytorch_dataset(ep_data, max_ep_length):
  padded_all_episode_feats = []
  padded_all_episode_labs = []
  for i in range(len(ep_data)):
    ep_feats = ep_data[i][0]
    ep_labs = ep_data[i][1]
    if len(ep_feats) != max_ep_length:
      num_zeros = max_ep_length - len(ep_feats)
      pad_feats_arr = np.zeros((num_zeros, ep_feats.shape[1]))
      pad_labs_arr = np.zeros((num_zeros, ep_labs.shape[1]))
      ep_feats = torch.cat((torch.tensor(pad_feats_arr, dtype=dtype, device=device), ep_feats), dim=0)
      ep_labs = torch.cat((torch.tensor(pad_labs_arr, dtype=dtype, device=device), ep_labs), dim=0)
      
    padded_all_episode_feats.append(ep_feats)
    padded_all_episode_labs.append(ep_labs)
  
  padded_pytorch_dataset = EpisodesDataset(padded_all_episode_feats, padded_all_episode_labs)
  return padded_pytorch_dataset

padded_train_dataset = ep_data_to_padded_pytorch_dataset(train_episodes, MAX_EP_LENGTH)
padded_valid_dataset = ep_data_to_padded_pytorch_dataset(valid_episodes, MAX_EP_LENGTH)
padded_test_dataset = ep_data_to_padded_pytorch_dataset(test_episodes, MAX_EP_LENGTH)

print(len(padded_train_dataset), len(padded_valid_dataset), len(padded_test_dataset))
padded_train_dataset[:2]

In [ ]:
from torch.nn.utils.rnn import pack_sequence

def my_collate(batch):
    # batch contains a list of tuples of structure (sequence, target)
    print(batch)
    data = [item[0] for item in batch]
    print(data)
    data = pack_sequence(data, enforce_sorted=False)
    print(data)
    targets = [item[1] for item in batch]
    print(targets)
    return [data, targets]


In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE=256

train_dataloader = DataLoader(padded_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(padded_valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(padded_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
out = lstm_model(train_features)
print(out.size())

In [ ]:
# try lstm to enable dependency between t, t+1 but only within the same episode
# train on gpu
from tqdm import tqdm

def lstm_train(train_eps, valid_eps, test_eps, model_, model_name, epochs=10000, learning_rate=0.1):
  criterion = nn.MSELoss()
  optimizer = torch.optim.SGD(model_.parameters(), lr=learning_rate)
  model_.name = model_name
  for epoch, _ in enumerate(tqdm(range(epochs))):
    start_time = time.time()
    train_loss_sum_sq = 0.
    for step, (ep_features, ep_labels) in enumerate(train_eps):
      optimizer.zero_grad()
      ep_preds = model_(ep_features)
      ep_loss = torch.sqrt(criterion(ep_preds, ep_labels))

      train_loss_sum_sq+=criterion(ep_preds, ep_labels).item()*len(ep_labels)

      ep_loss.backward()
      optimizer.step()
    if epoch % 1000 == 0:
      valid_loss_sum_sq = 0.
      for step, (ep_features, ep_labels) in enumerate(valid_eps):
        valid_preds = model_(ep_features)
        valid_loss_sum_sq +=criterion(valid_preds, ep_labels).item()*len(ep_labels)
      elapsed_time = time.time() - start_time
      print(f'epoch #{epoch} \n training loss: \n', np.sqrt(train_loss_sum_sq / len(train_eps)), \
          '\n validation loss: \n', np.sqrt(valid_loss_sum_sq / len(valid_eps)), '\n elapsed time: \n', elapsed_time)
  
  test_loss_sum_sq = 0.
  for ep_features, ep_labels in test_eps:
    test_preds = model_(ep_features)
    test_loss_sum_sq +=criterion(test_preds, ep_labels).item()*len(ep_labels)

  print('Final metrics: \n training loss: \n', np.sqrt(train_loss_sum_sq / len(train_eps)), \
          '\n validation loss: \n', np.sqrt(valid_loss_sum_sq / len(valid_eps)), \
        '\n test loss: \n', np.sqrt(test_loss_sum_sq / len(test_eps)))


In [ ]:
lstm_train(train_dataloader, valid_dataloader, test_dataloader, lstm_model, model_name='10000eps_LSTMnet_10000epochs' epochs=10000)

In [ ]:
sample = test_df.sample()
sample_feat = sample[features]
sample_lab = sample['time_to_terminate']
print('Linear model: ',regr.predict(sample_feat)[0])
print('Poisson model: ', pois_regr.predict(sample_feat)[0])
print('DenseNet model: ', dense_model(torch.tensor(sample_feat.values, dtype=dtype, device=device)).item())
print('DenseNetRelu model: ', dense_plus_relu_model(torch.tensor(sample_feat.values, dtype=dtype, device=device)).item())
print('LSTM model: ', lstm_model(torch.tensor(sample_feat.values.reshape(1,len(sample_feat), 8), dtype=dtype, device=device)).item())
print('Actual:  ', sample_lab.values[0])

In [ ]:
# align loss fn with others
# check ep loss fn -> maybe try ep loss for others instead of epoch loss

In [ ]:
# save all models

def save_models(model_list, dir):
  for model__ in model_list:
    model_file_name = f'{model__.name}.pt'
    torch.save(model__.state_dict(), dir+'/'+model_file_name)
  return ' models saved to directory ', dir

In [ ]:
!mkdir models

In [ ]:
save_models([dense_model, dense_plus_relu_model, lstm_model], 'models')

In [ ]:
# copy models to drive

from google.colab import drive

ROOT = '/content/drive'
PROJ = 'MyDrive/LearningPhysics/Models'
drive.mount(ROOT)


In [ ]:
from os.path import join

PROJECT_PATH = join(ROOT, PROJ)
print(PROJECT_PATH)

In [ ]:
!cd models
!cp -r /* {PROJECT_PATH}